<h1>Tensorflow: Single&Multiple Hidden Layer NNs

In [1]:
import tensorflow as tf
import numpy as np

<h3>產生資料集</h3>

In [2]:
np.random.seed(123)

Xtr = np.random.normal(0, 5, (2500, 3))
Xts = np.random.normal(0, 5, (2500, 3))

def f(x):
    y = (x[:,1] + 2*x[:,2] - x[:,0]  ## linear without interactions
         + x[:,0]*x[:,1] + x[:,0]*x[:,1]*x[:,2] ## linear with interactions
         #+ x[:,1]*(x[:,2]**2)  ## 非線性 有截距
        )
    return y.reshape(-1,1)

In [3]:
Ytr = f(Xtr)
Yts = f(Xts)

<h3>產生模型</h3>

超參數調整

In [12]:
## hyper paarameters
learning_rate = 0.01
n_nodes_1 = 32   ## 第一個隱藏層的神經元數
n_nodes_2 = 32   ## 第二個隱藏層的神經元數

架構 tensorflow

In [5]:
def create_weights(shape):
    ## 產生某一層神經層的權重
    ## 使用隨機常態分配初始權重，因為我們不想剛開始權重就過大或過小，故標準差(stddev)設為0.1， N(0, 0.1)
    inintializer = tf.random_normal(shape, stddev=0.1)
    w = tf.Variable(inintializer)
    return w

def create_biases(shape):
    ## 產生偏移
    ## 應該為 rank1 且 shape 要跟 n_nodes 一樣
    inintializer = tf.random_normal(shape)
    b = tf.Variable(inintializer)
    return b

In [13]:
n_input = Xtr.shape[1]
n_output = Ytr.shape[1]

# the 'None' allows us to pass different batch sizes
X = tf.placeholder(tf.float32, [None, n_input])
Y = tf.placeholder(tf.float32, [None, n_output])


## 輸入層
## 使用 dict 來整理權重
weights = {"w1": create_weights([n_input, n_nodes_1]),     ## 輸入到第一個隱藏層的神經權重
           "w2": create_weights([n_nodes_1, n_nodes_2]),     ## 第一個隱藏層到第二個隱藏層的神經權重
           "w_out": create_weights([n_nodes_2, n_output])} ## 輸出層的神經權重

## 使用 dict 來整理偏移
biases = {"b1": create_biases([n_nodes_1]),
          "b2": create_biases([n_nodes_2]),
          "b_out": create_biases([n_output])}

##  第一個隱藏層
z1 = tf.add(tf.matmul(X, weights["w1"]), biases["b1"])    ## 激活函數的 input
a1 = tf.nn.sigmoid(z1)  # 激活函數

##  第二個隱藏層
z2 = tf.add(tf.matmul(a1, weights["w2"]), biases["b2"])    ## 激活函數的 input
a2 = tf.nn.sigmoid(z2)  # 激活函數

## 輸出層
logits = tf.add(tf.matmul(a2, weights["w_out"]), biases["b_out"])
#yhat = tf.nn.softmax(logits)   ## 取得分類機率

## 反向傳播Backpropagation
# 定義 loss function : RMSE
loss = tf.sqrt(tf.reduce_mean(tf.square(Y-logits)))

# 定義優化器
#optimize = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
optimize = tf.train.RMSPropOptimizer(learning_rate).minimize(loss)

<h3>準備進行 train

In [14]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

batch_size = 100
n_epochs = 300
for epoch in range(1, n_epochs +1 ):
    # train on one batch at a time
    for i in range(0, len(Xtr), batch_size):
        sess.run(optimize, feed_dict={X: Xtr[i: i + batch_size],
                                      Y: Ytr[i: i + batch_size]})
    # 計算traing loss for printing progress
    if (epoch%10 == 0)| (epoch == 1):
        loss_tr = sess.run(loss, feed_dict={X: Xtr, Y: Ytr})
        print("Epoch {}, loss: {:.3f}".format(epoch, loss_tr, 3))

Epoch 1, loss: 116.021


Epoch 10, loss: 109.630


Epoch 20, loss: 95.007


Epoch 30, loss: 84.343


Epoch 40, loss: 75.356


...

Epoch 260, loss: 23.316


Epoch 270, loss: 22.837


Epoch 280, loss: 22.429


Epoch 290, loss: 21.692


Epoch 300, loss: 23.534


<h3>計算測試資料的 loss

In [15]:
loss_ts = sess.run(loss, feed_dict={X: Xts, Y: Yts})
logits_value = sess.run(logits, feed_dict={X: Xts})
print(10*"-")
print("Test loss %.3f"%np.round(loss_ts, 3))

----------
Test loss 37.365


In [16]:
logits_value.mean()

-0.57942235

In [17]:
Yts.mean()

-0.33574565054587663